In [1]:
from estnltk import Text
from estnltk.converters import export_TCF
from estnltk.converters import import_TCF

# TCF exporter and importer

Export `Text` object to TCF and import from `TCF`.
## Short example

In [2]:
# create a text object with segmentation layers
text = Text('Tere maailm!')
text.tag_layer('paragraphs')
text

Text(text='Tere maailm!')

In [3]:
# export the text object to a xml file and also print out the xml
# Notes:
# 1) paragraphs layer is not exported
# 2) TCF's tokens corresponds to EstNLTK's words. 
# EstNLTK's tokens and compound_tokens are auxiliary layers 
# and will not be exported
print(export_TCF(text, file='tcf/tcf_short.xml'))

<D-Spin xmlns="http://www.dspin.de/data" version="0.4">
  <MetaData xmlns="http://www.dspin.de/data/metadata"/>
  <TextCorpus xmlns="http://www.dspin.de/data/textcorpus" lang="et">
    <text>Tere maailm!</text>
    <tc:tokens xmlns:tc="http://www.dspin.de/data/textcorpus">
      <tc:token ID="t0" start="0" end="4">Tere</tc:token>
      <tc:token ID="t1" start="5" end="11">maailm</tc:token>
      <tc:token ID="t2" start="11" end="12">!</tc:token>
    </tc:tokens>
    <tc:sentences xmlns:tc="http://www.dspin.de/data/textcorpus">
      <tc:sentence ID="s0" tokenIDs="t0 t1 t2"/>
    </tc:sentences>
  </TextCorpus>
</D-Spin>



In [4]:
# The TCF importer reads an xml file and returns a text object.
import_TCF(file='tcf/tcf_short.xml')

Text(text='Tere maailm!')

## Longer example

In [5]:
from estnltk import Layer

text = Text('Karin, kes lendab New Yorki, tahab seal veeta puhkuse. Ta tuleb teisel augustil tagasi.')
text.tag_layer(['morph_analysis', 'paragraphs'])
text.pop_layer('tokens')

# clauses layer
layer = Layer(name='clauses', enveloping='words')
layer.add_annotation(text.words[2:6])
layer.add_annotation([text.words[0], *text.words[7:11]])
layer.add_annotation(text.words[12:17])
text.add_layer(layer)

# verb_chains layer
layer = Layer(name='verb_chains', enveloping='words')
layer.add_annotation(text.words[3:4])
layer.add_annotation(text.words[7:10:2])
layer.add_annotation(text.words[13:17:3])
text.add_layer(layer)

# time_phrases layer
layer = Layer(name='time_phrases', enveloping='words')
layer.add_annotation(text.words[14:16])
text.add_layer(layer)

text

Text(text='Karin, kes lendab New Yorki, tahab seal veeta puhkuse. Ta tuleb teisel augustil tagasi.')

In [6]:
text['sentences']

Layer(name='sentences', attributes=(), spans=SL[EnvelopingSpan(['Karin', ',', 'kes', 'lendab', 'New', 'Yorki', ',', 'tahab', 'seal', 'veeta', 'puhkuse', '.'], [{}]),
EnvelopingSpan(['Ta', 'tuleb', 'teisel', 'augustil', 'tagasi', '.'], [{}])])

In [7]:
text['clauses']

Layer(name='clauses', attributes=(), spans=SL[EnvelopingSpan(['Karin', 'tahab', 'seal', 'veeta', 'puhkuse'], [{}]),
EnvelopingSpan(['kes', 'lendab', 'New', 'Yorki'], [{}]),
EnvelopingSpan(['Ta', 'tuleb', 'teisel', 'augustil', 'tagasi'], [{}])])

In [8]:
text['verb_chains']

Layer(name='verb_chains', attributes=(), spans=SL[EnvelopingSpan(['lendab'], [{}]),
EnvelopingSpan(['tahab', 'veeta'], [{}]),
EnvelopingSpan(['tuleb', 'tagasi'], [{}])])

In [9]:
text['time_phrases']

Layer(name='time_phrases', attributes=(), spans=SL[EnvelopingSpan(['teisel', 'augustil'], [{}])])

The default version of the output of the TCF exporter is 0.4. This produces an xml document that can be used as as an input for [Weblicht service](https://weblicht.sfs.uni-tuebingen.de/weblicht/).

In [10]:
export_TCF(text, 'tcf/tcf.xml')
None

The output is in the file [tcf/tcf.xml](tcf/tcf.xml).

The version '0.5' produces an xml for possible future TCF version. This includes layers for clauses, verb_chains and time_phrases.

In [11]:
print(export_TCF(text, version='0.5'))

<D-Spin xmlns="http://www.dspin.de/data" version="0.5">
  <MetaData xmlns="http://www.dspin.de/data/metadata"/>
  <TextCorpus xmlns="http://www.dspin.de/data/textcorpus" lang="et">
    <text>Karin, kes lendab New Yorki, tahab seal veeta puhkuse. Ta tuleb teisel augustil tagasi.</text>
    <tc:tokens xmlns:tc="http://www.dspin.de/data/textcorpus">
      <tc:token ID="t0" start="0" end="5">Karin</tc:token>
      <tc:token ID="t1" start="5" end="6">,</tc:token>
      <tc:token ID="t2" start="7" end="10">kes</tc:token>
      <tc:token ID="t3" start="11" end="17">lendab</tc:token>
      <tc:token ID="t4" start="18" end="21">New</tc:token>
      <tc:token ID="t5" start="22" end="27">Yorki</tc:token>
      <tc:token ID="t6" start="27" end="28">,</tc:token>
      <tc:token ID="t7" start="29" end="34">tahab</tc:token>
      <tc:token ID="t8" start="35" end="39">seal</tc:token>
      <tc:token ID="t9" start="40" end="45">veeta</tc:token>
      <tc:token ID="t10" start="46" end="53">puhkuse</tc:t

## Tests for exporter and importer

In [12]:
text

Text(text='Karin, kes lendab New Yorki, tahab seal veeta puhkuse. Ta tuleb teisel augustil tagasi.')

In [13]:
# version 0.4
export_TCF(import_TCF(export_TCF(text))) == export_TCF(text)

True

In [14]:
#version 0.5
export_TCF(import_TCF(export_TCF(text, version='0.5')), version='0.5') == export_TCF(text, version='0.5')

True

In [15]:
# version 0.5
text.pop_layer('paragraphs')

text == import_TCF(export_TCF(text, version='0.5'))

True

In [16]:
#version 0.4
text.pop_layer('clauses')
text.pop_layer('verb_chains')
text.pop_layer('time_phrases')

text == import_TCF(export_TCF(text))

True